# A full business solution

## Now we will take our project from Day 1 to the next level

### BUSINESS CHALLENGE:

Create a product that builds a Brochure for a company to be used for prospective clients, investors and potential recruits.

We will be provided a company name and their primary website.

See the end of this notebook for examples of real-world business applications.

And remember: I'm always available if you have problems or ideas! Please do reach out.

In [1]:
# imports
# If these fail, please check you're running from an 'activated' environment with (llms) in the command prompt

import os
import requests
import json
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI

In [2]:
# Initialize and constants

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")
    
MODEL = 'gpt-4o-mini'
openai = OpenAI()

API key looks good so far


In [3]:
# A class to represent a Webpage

# Some websites need you to use proper headers when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:
    """
    A utility class to represent a Website that we have scraped, now with links
    """

    def __init__(self, url):
        self.url = url
        response = requests.get(url, headers=headers)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = ""
        links = [link.get('href') for link in soup.find_all('a')]
        self.links = [link for link in links if link]

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

In [4]:
ed = Website("https://edwarddonner.com")
ed.links

['https://edwarddonner.com/',
 'https://edwarddonner.com/outsmart/',
 'https://edwarddonner.com/about-me-and-about-nebula/',
 'https://edwarddonner.com/posts/',
 'https://edwarddonner.com/',
 'https://news.ycombinator.com',
 'https://nebula.io/?utm_source=ed&utm_medium=referral',
 'https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html',
 'https://patents.google.com/patent/US20210049536A1/',
 'https://www.linkedin.com/in/eddonner/',
 'https://edwarddonner.com/2024/12/21/llm-resources-superdatascience/',
 'https://edwarddonner.com/2024/12/21/llm-resources-superdatascience/',
 'https://edwarddonner.com/2024/11/13/llm-engineering-resources/',
 'https://edwarddonner.com/2024/11/13/llm-engineering-resources/',
 'https://edwarddonner.com/2024/10/16/from-software-engineer-to-ai-data-scientist-resources/',
 'https://edwarddonner.com/2024/10/16/from-software-engineer-to-ai-data-scientist-resources/',
 'https://edwarddonner.com/

## First step: Have GPT-4o-mini figure out which links are relevant

### Use a call to gpt-4o-mini to read the links on a webpage, and respond in structured JSON.  
It should decide which links are relevant, and replace relative links such as "/about" with "https://company.com/about".  
We will use "one shot prompting" in which we provide an example of how it should respond in the prompt.

This is an excellent use case for an LLM, because it requires nuanced understanding. Imagine trying to code this without LLMs by parsing and analyzing the webpage - it would be very hard!

Sidenote: there is a more advanced technique called "Structured Outputs" in which we require the model to respond according to a spec. We cover this technique in Week 8 during our autonomous Agentic AI project.

In [4]:
link_system_prompt = "You are provided with a list of links found on a webpage. \
You are able to decide which of the links would be most relevant to include in a brochure about the company, \
such as links to an About page, or a Company page, or Careers/Jobs pages.\n"
link_system_prompt += "You should respond in JSON as in this example:"
link_system_prompt += """
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}
"""

In [6]:
print(link_system_prompt)

You are provided with a list of links found on a webpage. You are able to decide which of the links would be most relevant to include in a brochure about the company, such as links to an About page, or a Company page, or Careers/Jobs pages.
You should respond in JSON as in this example:
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}



In [5]:
def get_links_user_prompt(website):
    user_prompt = f"Here is the list of links on the website of {website.url} - "
    user_prompt += "please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. \
Do not include Terms of Service, Privacy, email links.\n"
    user_prompt += "Links (some might be relative links):\n"
    user_prompt += "\n".join(website.links)
    return user_prompt

In [8]:
print(get_links_user_prompt(ed))

Here is the list of links on the website of https://edwarddonner.com - please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. Do not include Terms of Service, Privacy, email links.
Links (some might be relative links):
https://edwarddonner.com/
https://edwarddonner.com/outsmart/
https://edwarddonner.com/about-me-and-about-nebula/
https://edwarddonner.com/posts/
https://edwarddonner.com/
https://news.ycombinator.com
https://nebula.io/?utm_source=ed&utm_medium=referral
https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html
https://patents.google.com/patent/US20210049536A1/
https://www.linkedin.com/in/eddonner/
https://edwarddonner.com/2024/12/21/llm-resources-superdatascience/
https://edwarddonner.com/2024/12/21/llm-resources-superdatascience/
https://edwarddonner.com/2024/11/13/llm-engineering-resources/
https://edwarddonner.com/2024/11/13/ll

In [6]:
def get_links(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(website)}
      ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    return json.loads(result)

In [10]:
# Anthropic has made their site harder to scrape, so I'm using HuggingFace..

huggingface = Website("https://huggingface.co")
huggingface.links

['/',
 '/models',
 '/datasets',
 '/spaces',
 '/posts',
 '/docs',
 '/enterprise',
 '/pricing',
 '/login',
 '/join',
 '/deepseek-ai/DeepSeek-V3',
 '/deepseek-ai/DeepSeek-V3-Base',
 '/PowerInfer/SmallThinker-3B-Preview',
 '/black-forest-labs/FLUX.1-dev',
 '/hexgrad/Kokoro-82M',
 '/models',
 '/spaces/JeffreyXiang/TRELLIS',
 '/spaces/osanseviero/gemini-coder',
 '/spaces/lllyasviel/iclight-v2',
 '/spaces/Kwai-Kolors/Kolors-Virtual-Try-On',
 '/spaces/Qwen/QVQ-72B-preview',
 '/spaces',
 '/datasets/agibot-world/AgiBotWorld-Alpha',
 '/datasets/fka/awesome-chatgpt-prompts',
 '/datasets/PowerInfer/QWQ-LONGCOT-500K',
 '/datasets/HuggingFaceTB/finemath',
 '/datasets/O1-OPEN/OpenO1-SFT',
 '/datasets',
 '/join',
 '/pricing#endpoints',
 '/pricing#spaces',
 '/pricing',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/allenai',
 '/facebook',
 '/amazon',
 '/google',
 '/Intel',
 '/microsoft',
 '/grammarly',
 '/Writer',
 '/docs/transformers',

In [11]:
result = get_links("https://huggingface.co")
print(result)

{'links': [{'type': 'about page', 'url': 'https://huggingface.co/huggingface'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'}, {'type': 'documentation page', 'url': 'https://huggingface.co/docs'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'community discussion page', 'url': 'https://discuss.huggingface.co'}, {'type': 'GitHub page', 'url': 'https://github.com/huggingface'}, {'type': 'LinkedIn page', 'url': 'https://www.linkedin.com/company/huggingface'}]}


## Second step: make the brochure!

Assemble all the details into another prompt to GPT4-o

In [7]:
def get_all_details(url):
    result = "Landing page:\n"
    result += Website(url).get_contents()
    links = get_links(url)
    print("Found links:", links)
    for link in links["links"]:
        result += f"\n\n{link['type']}\n"
        result += Website(link["url"]).get_contents()
    return result

In [13]:
print(get_all_details("https://huggingface.co"))

Found links: {'links': [{'type': 'home page', 'url': 'https://huggingface.co'}, {'type': 'about page', 'url': 'https://huggingface.co/huggingface'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'community discussions', 'url': 'https://discuss.huggingface.co'}, {'type': 'GitHub page', 'url': 'https://github.com/huggingface'}, {'type': 'LinkedIn page', 'url': 'https://www.linkedin.com/company/huggingface/'}, {'type': 'Twitter page', 'url': 'https://twitter.com/huggingface'}]}
Landing page:
Webpage Title:
Hugging Face – The AI community building the future.
Webpage Contents:
Hugging Face
Models
Datasets
Spaces
Posts
Docs
Enterprise
Pricing
Log In
Sign Up
The AI community building the future.
The platform where the machine learning community collaborate

In [8]:
system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
and creates a short brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
Include details of company culture, customers and careers/jobs if you have the information."

# Or uncomment the lines below for a more humorous brochure - this demonstrates how easy it is to incorporate 'tone':

# system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
# and creates a short humorous, entertaining, jokey brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
# Include details of company culture, customers and careers/jobs if you have the information."


In [9]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"You are looking at a company called: {company_name}\n"
    user_prompt += f"Here are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\n"
    user_prompt += get_all_details(url)
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    return user_prompt

In [16]:
get_brochure_user_prompt("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/huggingface'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'}, {'type': 'blog', 'url': 'https://huggingface.co/blog'}, {'type': 'discussion page', 'url': 'https://discuss.huggingface.co'}, {'type': 'GitHub page', 'url': 'https://github.com/huggingface'}, {'type': 'Twitter page', 'url': 'https://twitter.com/huggingface'}, {'type': 'LinkedIn page', 'url': 'https://www.linkedin.com/company/huggingface/'}]}


'You are looking at a company called: HuggingFace\nHere are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\nLanding page:\nWebpage Title:\nHugging Face – The AI community building the future.\nWebpage Contents:\nHugging Face\nModels\nDatasets\nSpaces\nPosts\nDocs\nEnterprise\nPricing\nLog In\nSign Up\nThe AI community building the future.\nThe platform where the machine learning community collaborates on models, datasets, and applications.\nTrending on\nthis week\nModels\ndeepseek-ai/DeepSeek-V3\nUpdated\n4 days ago\n•\n54.8k\n•\n1.03k\ndeepseek-ai/DeepSeek-V3-Base\nUpdated\n4 days ago\n•\n7.53k\n•\n1.11k\nPowerInfer/SmallThinker-3B-Preview\nUpdated\n1 day ago\n•\n1.44k\n•\n178\nblack-forest-labs/FLUX.1-dev\nUpdated\nAug 16, 2024\n•\n1.17M\n•\n7.69k\nhexgrad/Kokoro-82M\nUpdated\nabout 20 hours ago\n•\n449\n•\n165\nBrowse 400k+ models\nSpaces\nRunning\non\nZero\n2.46k\n🏢\nTRELLIS\nScalable and Versati

In [10]:
def create_brochure(company_name, url):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [18]:
create_brochure("HuggingFace", "https://huggingface.com")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.com/huggingface'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'blog', 'url': 'https://huggingface.com/blog'}, {'type': 'enterprise page', 'url': 'https://huggingface.com/enterprise'}, {'type': 'pricing page', 'url': 'https://huggingface.com/pricing'}, {'type': 'community page', 'url': 'https://discuss.huggingface.co'}, {'type': 'GitHub page', 'url': 'https://github.com/huggingface'}, {'type': 'LinkedIn page', 'url': 'https://www.linkedin.com/company/huggingface/'}, {'type': 'Twitter page', 'url': 'https://twitter.com/huggingface'}]}


# Hugging Face Company Brochure

**Welcome to Hugging Face!**  
*The AI community building the future.*

---

## About Us

Hugging Face is at the forefront of the machine learning revolution, serving as a collaborative platform where the AI community can create, discover, and improve upon models, datasets, and applications. With a strong commitment to democratizing access to machine learning, Hugging Face provides tools and resources that empower developers, researchers, and enthusiasts to contribute their ideas, enhancing the overall ecosystem of AI.

## Our Offerings

- **Models**: Explore and collaborate on over 400,000 models, including cutting-edge technologies like Transformers and Diffusers for a variety of applications - from text generation to image synthesis.

- **Datasets**: Access and share a diverse range of 100,000+ datasets to support your machine learning projects.

- **Spaces**: Create and run applications seamlessly with our user-friendly interface. Hugging Face provides a home for more than 150,000 applications tailored for real-world use.

- **Enterprise Solutions**: For organizations looking to leverage AI at scale, we offer enterprise-grade security, dedicated support, and advanced compute capabilities, starting at just $20/user/month.

## Our Customers

Join over **50,000 organizations** that have adopted Hugging Face as their machine learning platform, including industry giants like:

- **Meta**
- **Amazon Web Services**
- **Google**
- **Microsoft**
- **Intel**

---

## Company Culture

At Hugging Face, we foster an open and collaborative environment. Our mission is driven by a strong commitment to community involvement and transparency in AI. We believe that everyone should have access to the tools necessary for building the future of machine learning, and every team member plays an essential role in this vision.

### Our Values:

- **Community**: We enrich the ecosystem through collaboration and open-source contributions.
- **Innovation**: Continuous improvement and cutting-edge development are at the heart of our practices.
- **Inclusivity**: We strive to create an environment where every voice matters.

---

## Careers at Hugging Face

We are always on the lookout for passionate individuals to join our team! Whether you are interested in engineering, product management, research, or community engagement, there’s a place for you at Hugging Face. 

### Why Join Us?

- **Impactful Work**: Contribute to projects that shape the future of AI.
- **Learning Environment**: Work alongside experienced professionals and grow your skills in a collaborative setting.
- **Diversity and Inclusion**: We welcome talents from various backgrounds and encourage a culture of diversity.

---

## Get In Touch

We invite you to become a part of our vibrant community! Explore our platform, collaborate on models, datasets, and applications, and help us build the future of AI together.

- **Website**: [Hugging Face](https://huggingface.co)
- **Follow us on Social Media**: [GitHub](https://github.com/huggingface), [Twitter](https://twitter.com/huggingface), [LinkedIn](https://www.linkedin.com/company/huggingface)

Let's innovate and accelerate machine learning, one commit at a time!

---

In [11]:
create_brochure("Contextual AI","https://contextual.ai/")

Found links: {'links': [{'type': 'company page', 'url': 'https://contextual.ai/company/'}, {'type': 'careers page', 'url': 'https://contextual.ai/careers'}]}


```markdown
# Contextual AI Brochure

## Welcome to Contextual AI

At **Contextual AI**, we are at the forefront of building production-ready Enterprise AI applications, specializing in Retrieval-Augmentation Generation (RAG). Our innovative technology empowers businesses to harness the full potential of their data, turning vast amounts of information into actionable insights smoothly and efficiently.

### Our Mission

We strive to deliver highly accurate, auditable, and specialized AI solutions tailored to meet the unique needs of our customers. Contextual AI is not just about technology; we are dedicated to transforming how enterprises operate by making AI accessible and effective.

---

## Industry Leadership 

As pioneers of RAG, we have successfully deployed AI systems for Fortune 500 companies and other industry leaders, managing **tens of thousands of documents** and millions of pages. Our clients span various critical sectors including:

- **Financial Services**: Enhancing compliance, analysis, and reporting.
- **Technology & Engineering**: Streamlining R&D and engineering processes.
- **Professional Services**: Boosting productivity in legal and advisory firms.

### Key Advantages of Contextual AI

- **Production-Ready Accuracy**: Powered by RAG 2.0, we deliver unparalleled accuracy and speed, allowing applications to be tuned and deployed rapidly.
  
- **Fast to Deploy, Easy to Maintain**: Our platform can be operational within minutes, bypassing lengthy installation and integration processes.

- **Enterprise-Grade Security and Controls**: Meeting stringent security requirements, our solutions are built to ensure data protection and compliance.

- **Flexible and Scalable Deployment**: Choose from managed SaaS, your VPC, or on-premises solutions, maintaining complete control over your data.

---

## Company Culture

At Contextual AI, we foster a culture of innovation, collaboration, and continuous learning. Our team consists of **RAG and Machine Learning experts** who are passionate about developing cutting-edge technology that meets the real-world needs of our clients. We believe in the importance of feedback and adaptability, encouraging our workforce to explore and create solutions that push boundaries.

### Join Our Team

We are always looking for talented individuals to join our growing workforce. If you're eager to be part of a dynamic company that values employee input and fosters growth through learning, consider applying to Contextual AI. Check our [Careers](#) page for current opportunities.

---

## Connect With Us!

Interested in learning more about our offerings? Contact us today for detailed information and to request access to our platform. 

Stay updated with our latest news and research by following us on [LinkedIn](#) and [Twitter](#).

---

**Contextual AI: Turning AI Vision into Action.**

> _Transform your knowledge into actionable intelligence with Contextual AI._
```


## Finally - a minor improvement

With a small adjustment, we can change this so that the results stream back from OpenAI,
with the familiar typewriter animation

In [12]:
def stream_brochure(company_name, url):
    stream = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
        stream=True
    )
    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

In [20]:
stream_brochure("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/huggingface'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'}, {'type': 'blog', 'url': 'https://huggingface.co/blog'}, {'type': 'models page', 'url': 'https://huggingface.co/models'}, {'type': 'datasets page', 'url': 'https://huggingface.co/datasets'}, {'type': 'spaces page', 'url': 'https://huggingface.co/spaces'}, {'type': 'documentation page', 'url': 'https://huggingface.co/docs'}]}


# Hugging Face Brochure

## Introduction
**Hugging Face** is a cutting-edge platform that serves as a hub for the artificial intelligence (AI) community, dedicated to fostering collaboration among machine learning enthusiasts and professionals. With a mission to democratize and simplify access to machine learning tools, Hugging Face provides a suite of models, datasets, and spaces to enhance the development and deployment of AI applications.

## Our Vision
**"The AI community building the future."**

Hugging Face aims to create an inclusive environment where users can discover, create, and collaborate on machine learning resources, empowering developers and researchers to push the boundaries of AI technology.

## Features

### Models
- Access over **400,000 models**, including cutting-edge solutions for natural language processing, computer vision, and more.
- Collaborate on model development and updates, enhancing your AI projects efficiently.

### Datasets
- Browse through a comprehensive collection of **100,000+ datasets** to find the perfect match for your machine learning needs.

### Spaces
- Engage with a selection of **150,000+ applications** through dedicated spaces where users can showcase and share their work, facilitating innovation and experimentation.

### Open Source Commitment
- Hugging Face is devoted to advancing **open-source machine learning tooling** through collaborative projects like Transformers, Diffusers, and more, which are essential for both research and production.

## Community & Culture
At Hugging Face, we believe in the power of community. Our culture is driven by:
- **Collaboration:** We provide platforms for developers, researchers, and enthusiasts to come together and contribute.
- **Innovation:** We encourage creativity and exploration, allowing community members to push the envelope of AI capabilities.
- **Inclusion:** Our aim is to make machine learning accessible for everyone, regardless of their background or expertise level.

### Our Users
More than **50,000 organizations** rely on Hugging Face, including industry leaders such as:
- **Amazon Web Services**
- **Google**
- **Microsoft**

These partnerships affirm our standing as a trusted resource for enterprise-grade AI solutions.

## Careers at Hugging Face
Join our passionate team! We're looking for innovators who are excited about machine learning and eager to contribute to an open and collaborative environment. We offer opportunities across various roles in engineering, research, and support.

**Explore careers with us:** [Hugging Face Careers](https://huggingface.co/jobs)

## Get Involved Today!
Whether you are a developer looking to leverage our tools, a company interested in enterprise solutions, or a job seeker eager to join a pioneering team, Hugging Face invites you to be part of our journey towards building a better AI future.

**Sign Up or Learn More at:** [Hugging Face](https://huggingface.co)  
**Connect with Us:** [Twitter](https://twitter.com/huggingface) | [GitHub](https://github.com/huggingface) | [LinkedIn](https://linkedin.com/company/huggingface)

---

Unleash your potential with **Hugging Face** – where the machine learning community thrives!

In [22]:
# Try changing the system prompt to the humorous version when you make the Brochure for Hugging Face:

stream_brochure("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'company page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'community discussion', 'url': 'https://discuss.huggingface.co'}, {'type': 'GitHub page', 'url': 'https://github.com/huggingface'}, {'type': 'LinkedIn page', 'url': 'https://www.linkedin.com/company/huggingface/'}, {'type': 'Twitter page', 'url': 'https://twitter.com/huggingface'}]}


# Welcome to Hugging Face! 🤗

### The AI Community Building the Future

---

**Are you tired of lonely algorithms? Join Hugging Face where ML models get a makeover, datasets find their match, and applications get to mingle!** 

---

### Why Hugging Face? 

**We aren't just a tech company; we're an AI community!** Think of us as the matchmakers of the machine learning world—supercharging collaborations that lead to happiness (and better models).

---

### What Do We Offer? 🌟

- **400,000+ Models:** Why settle for one when you can have a whole buffet? From deep learning to transformers, we have them all!  
- **150,000+ Applications:** Want to model your dress sense? Our virtual try-ons have you covered.  
- **100,000+ Datasets:** Because every model deserves a great backstory.

---

### Trending This Week 🔥

- **DeepSeek-V3:** 54.8k updates and growing! Definitely a model with 'friends.'  
- **FLUX.1-dev:** Over 1.17M followers—defining what it means to be the “model” citizen!  

---

### Who Loves Us? 🥰

Join over **50,000 organizations** (including giant techies like Amazon, Google, and Microsoft) who are giving us the perfect five-star rating on their enterprise-scale endeavors. For the uninitiated, that’s kind of a big deal!

---

### Careers with Hugging Face 🚀

**Join our team where the culture is as diverse as our datasets!** Can't tell your Transformers from your Tokenizers? Don't worry; we're here to train you. 

- **What You’ll Enjoy:**
  - A collaborative atmosphere where you can flex those AI muscles.
  - Plenty of opportunities to upgrade your career (just like our models)!
  - Snacks that are more diverse than your daily data inputs.

---

### Why Use Hugging Face? 🎉

- **Open Source for All:** Yes, that means you heard right! We are giving away tools that make machine learning a walk in the park. 
- **Compute from A to Z:** Forget about complicated GPU setups. With us, it starts at a mere **$0.60/hour**. (Serious savings if your budget's screaming!)
- **Enterprise Packages:** Perfect for billion-dollar ideas that need the nurturing they deserve. 

---

### Ready to Join the Fun?

Whether you're a customer looking for AI solutions, an investor seeking the next big break, or a future recruit wanting to dive into an exciting career, **Hugging Face is your destination!**

**After all, who wouldn't want to be a part of a community that celebrates hugs, algorithms, and a whole lot of creativity?! 😄**

### Connect with Us!
- **Website:** [huggingface.co](https://huggingface.co)
- **Twitter:** [@HuggingFace](https://twitter.com/huggingface)
- **LinkedIn:** [Hugging Face](https://www.linkedin.com/company/huggingface)

---

Let’s transform the future together! Come, give us a hug (virtually, of course!).

In [13]:
stream_brochure("Contextual AI","https://contextual.ai/")

Found links: {'links': [{'type': 'company page', 'url': 'https://contextual.ai/company/'}, {'type': 'careers page', 'url': 'https://contextual.ai/careers'}]}



# Contextual AI Brochure

## About Contextual AI

**Build Production-Grade Enterprise RAG Applications**

Contextual AI is a pioneer in developing production-ready artificial intelligence (AI) applications tailored for enterprise needs, specifically in the context of Retrieval-Augmented Generation (RAG). With a commitment to providing accurate, auditable, and secure AI solutions, we empower organizations across various sectors to harness the power of data efficiently.

### Our Offerings

- **Production-Ready Applications**: Our advanced RAG platform enables businesses to build and deploy applications with high accuracy and rapid deployment cycles, enhancing productivity and enabling data-driven insights.
  
- **Industry-Specific Solutions**:
  - **Financial Services**: Helping teams in wealth management and capital markets access critical information.
  - **Technology & Engineering**: Streamlining processes by transforming technical documentation into actionable knowledge.
  - **Professional Services**: Enhancing productivity in legal and advisory services with powerful document synthesis and search capabilities.

### Why Choose Us?

- **Proven Track Record**: Contextual AI has successfully deployed solutions for Fortune 500 companies, showcasing our capability to handle large-scale, complex document processing tasks.
- **Rapid Deployment**: Our platform allows for quick setup and maintenance, minimizing the time spent on development.
- **Enterprise-Grade Security**: Built with robust security and compliance features, our solutions meet the stringent requirements of regulated industries.
- **Flexible Deployment Options**: Options for fully managed SaaS, Virtual Private Cloud (VPC), or on-premises setups ensure that your deployment aligns with your operational needs.

### Who We Serve

Our esteemed clientele spans across regulated and security-conscious industries, including:
- Large enterprises in **internal and external financial services**.
- Leading **technology and engineering firms**.
- Innovative **professional services** organizations.

### Company Culture

At Contextual AI, we value innovation, collaboration, and integrity. Our team comprises experts in AI and machine learning who are dedicated to delivering cutting-edge solutions while fostering an inclusive and dynamic workplace culture. We encourage continuous learning, professional growth, and teamwork that drives success.

### Careers at Contextual AI

We are actively seeking talented individuals to join our team. If you are passionate about shaping the future of AI and want to work in an environment that promotes development and success, explore our career opportunities:

- Positions available across engineering, data science, product management, and more.
- Ongoing training programs and professional development initiatives.
- A collaborative and forward-thinking work environment.

Join us in redefining enterprise AI solutions!

### Get In Touch

Interested in learning more about our solutions, careers, or how we can assist your organization? 
- **Contact Us**: [Contact Information Link]
- **Follow Us**: Connect with us on [LinkedIn](#) and [Twitter](#).

---

Empower your business with Contextual AI, where the future of enterprise intelligence begins!



<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../business.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#181;">Business applications</h2>
            <span style="color:#181;">In this exercise we extended the Day 1 code to make multiple LLM calls, and generate a document.

This is perhaps the first example of Agentic AI design patterns, as we combined multiple calls to LLMs. This will feature more in Week 2, and then we will return to Agentic AI in a big way in Week 8 when we build a fully autonomous Agent solution.

Generating content in this way is one of the very most common Use Cases. As with summarization, this can be applied to any business vertical. Write marketing content, generate a product tutorial from a spec, create personalized email content, and so much more. Explore how you can apply content generation to your business, and try making yourself a proof-of-concept prototype.</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../important.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#900;">Before you move to Week 2 (which is tons of fun)</h2>
            <span style="color:#900;">Please see the week1 EXERCISE notebook for your challenge for the end of week 1. This will give you some essential practice working with Frontier APIs, and prepare you well for Week 2.</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../resources.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#f71;">A reminder on 2 useful resources</h2>
            <span style="color:#f71;">1. The resources for the course are available <a href="https://edwarddonner.com/2024/11/13/llm-engineering-resources/">here.</a><br/>
            2. I'm on LinkedIn <a href="https://www.linkedin.com/in/eddonner/">here</a> and I love connecting with people taking the course!
            </span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../thankyou.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#090;">Finally! I have a special request for you</h2>
            <span style="color:#090;">
                My editor tells me that it makes a MASSIVE difference when students rate this course on Udemy - it's one of the main ways that Udemy decides whether to show it to others. If you're able to take a minute to rate this, I'd be so very grateful! And regardless - always please reach out to me at ed@edwarddonner.com if I can help at any point.
            </span>
        </td>
    </tr>
</table>

In [24]:
stream_brochure('Maharshi Industries', 'https://www.maharshi.com/')

Found links: {'links': [{'type': 'main website', 'url': 'https://www.maharshi.com'}, {'type': 'company overview', 'url': 'https://www.maharshi.com/company-overview'}, {'type': 'ethos and growth journey', 'url': 'https://www.maharshi.com/ethos-growth-journey'}, {'type': 'awards and milestones', 'url': 'https://www.maharshi.com/awards-milestone'}, {'type': 'presence', 'url': 'https://www.maharshi.com/presence'}, {'type': 'careers page', 'url': 'https://www.maharshi.com/working-at-maharshi'}, {'type': 'search all opportunities', 'url': 'https://www.maharshi.com/search-all-opportunities'}, {'type': 'newsroom', 'url': 'https://www.maharshi.com/newsroom'}, {'type': 'blog', 'url': 'https://www.maharshi.com/blog'}]}



# Maharshi Industries Brochure

## About Us
**Maharshi Industries** – A Leader in Labelling & Packaging Solutions

Established in **1980**, Maharshi Industries is headquartered in **Ahmedabad, Gujarat, India**. Over four decades, we have built an expansive reputation for innovation in various sectors, including pharmaceuticals, breweries, cosmetics, and food processing. Our commitment to excellence has allowed us to emerge as a **multi-product, multi-location engineering group** with expertise in the design, development, and manufacturing of advanced labelling and packaging machinery.

## Our Products
At Maharshi, we pride ourselves on offering an extensive range of products tailored to meet diverse industrial needs:
- **Labelling Machines**: Including Rotary Sticker, Double-Side Sticker, and High-Speed Ampoule/Vial Labelling Machines.
- **Packaging Machines**: Various liquid filling machines designed for different viscosities, including Level Filling, Viscous/Non-Viscous Liquid Filling, and Electronic Liquid Filling Machines.
- **Packaging Lines**: Comprehensive solutions for product types, designed to enhance productivity through seamless integration.

Our machinery is engineered to ensure **100% accuracy**, reliability, and compliance with international **GMP standards**.

## Global Presence
With a footprint in **45+ countries** including:
- Bangladesh
- Australia
- USA
- Germany
- South Africa
- Vietnam
- Egypt

Our solutions have been trusted worldwide, enabling us to cater to a global clientele.

## Company Culture
At Maharshi Industries, we foster a culture of **partnership, precision, and proactiveness**. Our work environment encourages innovation, teamwork, and continuous improvement. We recognize and value our employees, understanding that their growth and satisfaction drive our collective success. 

### Careers at Maharshi
We are constantly on the lookout for passionate individuals who thrive in a collaborative environment. Exploring a career at Maharshi means being part of a dynamic team that values innovation and excellence. With a variety of roles available, we provide ample opportunities for professional development.

Discover our current job openings and consider joining us in our quest for excellence.

## Contact Us
For inquiries, support or to learn more about opportunities, please reach out at:
- **Email**: info@maharshi.com

---

Join us on our journey as we continue to redefine the labelling and packaging landscape. Maharshi Industries: **Where Innovation Meets Excellence**.


In [25]:
stream_brochure('Phlipton','https://phlipton.com/')

Found links: {'links': [{'type': 'about page', 'url': 'https://phlipton.com/about-us/'}, {'type': 'products page', 'url': 'https://phlipton.com/products/'}, {'type': 'shop page', 'url': 'https://phlipton.com/shop/'}, {'type': 'careers page', 'url': 'https://phlipton.com/contact-us/'}, {'type': 'company social media', 'url': 'https://www.facebook.com/Phlipton'}, {'type': 'company social media', 'url': 'https://twitter.com/phlipton'}, {'type': 'company social media', 'url': 'https://www.instagram.com/phliptonautomation/'}]}


# Welcome to Phlipton Automation

**Flip The Way You Switch!**  
Phlipton is your go-to automation solutions provider, dedicated to transforming homes and businesses in India with cutting-edge smart technology. Our motto revolves around making automation accessible to everyone, and our product offerings ensure seamless integration between everyday tasks and advanced automation.

---

## 🌟 About Us
Founded with the vision to simplify technology, Phlipton stands out as **India’s Fastest Growing Smart Home and Industrial Automation** company. We believe in making technology affordable and manageable for all income groups, thereby enabling a smarter way of living.

- **Empowering Automation:** Phlipton combines **IoT, machine-to-machine connections,** and smart systems to empower users with complete control. 
- **Remote Monitoring:** Our solutions allow users to remotely monitor their surroundings, providing peace of mind and convenience at all times.

---

## 💡 Our Products
At Phlipton, we offer a wide range of products designed with both aesthetics and functionality in mind. Our offerings include:

- **Smart Switches & Control:** Modern switches that integrate with your lifestyle.
- **Smart Lighting:** Customizable lighting solutions to enhance ambiance.
- **Smart Locks & Security Systems:** Features that ensure safety and comfort.
- **Hotel Automation Solutions:** Tailored for the hospitality industry.
- **Virtual Voice Assistant Devices:** Manage your environment using just your voice!

Explore our product packages tailored to fit your home or office needs, with the option to customize further to suit your preferences.

---

## 👥 Our Team
Our team is comprised of dedicated professionals with a shared commitment to innovation and customer satisfaction:
- **Jay Patel** - Managing Director
- **Neel Sathwara** - Business Development

---

## 🎉 Company Culture
Phlipton fosters an inclusive and innovative culture, where teamwork and collaboration empower our employees to explore and bring forth new ideas. We're committed to:
- **Continuous Learning:** Opportunities for personal and professional growth.
- **Innovation:** Encouraging creative solutions for complex automation challenges.
- **Customer-Centric Approach:** Every decision is made with the customer in mind.

---

## 📈 Careers at Phlipton
Join our team! We are constantly on the lookout for enthusiastic individuals eager to contribute to the automation revolution. Available roles span across development, marketing, and customer service sectors. At Phlipton, you’ll find opportunities to grow and work with pioneering technology. 

---

## 📞 Get in Touch
Ready to upscale your home or business with Phlipton? Let’s start the conversation!  
- **Phone:** +919624733006  
- **Email:** info@phlipton.com  
- **Address:** A1004, Shree Ganesh Glory 11, Jagatpur, SG Highway, Ahmedabad

Explore the potential of automation. **[Schedule a Free Demo Today!](#)**

---

**Phlipton Automation – Where Technology Meets Simplicity!**  
**Join Us, Flip the way you switch, and embrace the future of automation!**

In [26]:
stream_brochure('Refactor Guru','https://refactoring.guru/')

Found links: {'links': [{'type': 'about page', 'url': 'https://refactoring.guru/site-about'}, {'type': 'company page', 'url': 'https://refactoring.guru'}, {'type': 'careers page', 'url': 'https://github.com/RefactoringGuru'}, {'type': 'design patterns page', 'url': 'https://refactoring.guru/design-patterns'}, {'type': 'refactoring page', 'url': 'https://refactoring.guru/refactoring'}]}


# Refactor Guru Company Brochure

---

## About Refactor Guru

**Refactoring.Guru** is the premier online resource dedicated to refining your programming skills through a deeper understanding of **refactoring**, **design patterns**, **SOLID principles**, and various other essential programming concepts. Established by Alexander Shvets, Refactor Guru aims to illuminate the dynamic connections between these concepts, making complex topics accessible to developers at all levels.

### Company Mission

At Refactor Guru, our mission is to simplify complex programming techniques and bridge the knowledge gap around refactoring and design patterns. We strive to provide comprehensive information that empowers programmers to produce clean, maintainable, and efficient code.

---

## Our Services

- **Refactoring Guides:** Learn the systematic processes of improving code without altering its functionality. Explore topics like code smells, when and how to refactor, and much more.
- **Design Patterns:** Delve into typical solutions for common software design problems through detailed descriptions and real-world examples of design patterns.
- **Premium Content:** Unlock exclusive resources such as our Design Patterns eBook and comprehensive refactoring courses to take your learning to the next level.

---

## Company Culture

Refactor Guru fosters an environment of continuous learning and community engagement. Being a one-man band doesn't limit our outreach; Alexander Shvets actively engages with users via emails and social media, ensuring an interactive experience. Our commitment to delivering quality content without spam reflects our respect for our users’ time and attention.

---

## Our Customers

We cater to a diverse audience, including:
- **Professional Developers:** Seeking to enhance their skills and improve their code quality through advanced programming techniques.
- **Students:** Eager to learn the foundations of programming and refine their coding abilities.
- **Educators:** Looking for resources and tools to teach design patterns and refactoring principles in classrooms.

---

## Careers at Refactor Guru

Although we are currently a small team led by Alexander Shvets, Refactor Guru welcomes passionate individuals who share our vision of improving programming education. As we grow, potential job openings will be available across various areas such as content creation, community management, and software development.

---

## Connect with Us

Stay updated with our latest offerings and content by subscribing to our email list, where we promise no more than one email per month. Join our growing community on [Facebook](https://facebook.com) and connect with other programming enthusiasts.

---

## Join Us in the Journey

Whether you're looking to refine your skills, learn new design patterns, or simply discover the intersection of clean coding practices, **Refactor Guru** is here to support you every step of the way. Visit us today for a wealth of resources designed to empower programmers worldwide.

### Shop Now!

Explore our premium content and dive into your next programming adventure!

--- 

**Refactor Guru**  
All rights reserved 2014-2025  
[Contact Us](mailto:contact@refactoring.guru)  
[Visit our Website](https://refactoring.guru)  


# Graphical Interface using Gradio
 Now applying gui to make a application that takes model, wordlength, url and tone as input and give markdown brochure as output.

In [19]:
import gradio as gr
import ollama

In [38]:
# def get_brochure_prompt(url, tone, wordlength):
def get_brochure_prompt(url, tone):
    system_prompt = 'You are an expert brochure writer, your job to is to provide me a detailed and extensive company brochure.\n'
    system_prompt += 'The brochure should be informative which I can share with my investors, recruits and interested people.\n'
    system_prompt += f'Response should be well formatted markdown with appropriate titles and the tone of brochure should be {tone}\n'
    system_prompt += f'Dont hallucinate if you dont get relevant company data, say that you cannot make a brocure due to lack of info about the company.\n'

    user_prompt = f'Help me writing a brochure for a company titled - {Website(url).title}.\n'
    user_prompt = f'Give response as markdown without writing `markdown` as header of response.\n'
    # user_prompt += f'Keep note of the desired tone and the brochure length should be around {wordlength}.\n'
    user_prompt += f'Below is the data I scraped from the website homepage and other relevant pages\n:{get_all_details(url)}'

    return [
        {
            'role':'system',
            'content':system_prompt
        },
        {
            'role':'user',
            'content':user_prompt
        },
    ]
    

In [21]:
def stream_gpt(messages):
    stream = openai.chat.completions.create(
        model='gpt-4o-mini',
        messages=messages,
        stream=True
    )
    result = ''
    for chunk in stream:
        result += chunk.choices[0].delta.content or ''
        yield result    

In [34]:
def stream_opensource(model, messages):
    stream = ollama.chat(
        model=model,
        messages=messages,
        stream=True
    )

    result = ''
    for chunk in stream:
        result += chunk['message']['content'] or ''
        yield result

In [35]:
# def stream_model(url, model, tone, wordlength):
def stream_model(url, model, tone):
    # messages = get_brochure_prompt(url, tone, wordlength)
    messages = get_brochure_prompt(url, tone)
    if model == 'ChatGpt 4o-mini':
        yield from stream_gpt(messages)
    elif model in ['Mistral', 'Qwen2.5', 'llama3.2']:
        model = model.lower()
        yield from stream_opensource(model=model, messages=messages)
    else:
        raise ValueError('Invalid model choice')

In [14]:
print(stream_model('https://www.gradio.app/guides/quickstart', 'ChatGpt 4o-mini', 'funny', 1000))

<generator object stream_model at 0x00000288F9FF8C40>


In [40]:
# view = gr.Interface(
#     fn=stream_model,
#     inputs=[
#         gr.Textbox(label='Paste the complete company url here:'),
#         gr.Dropdown(['ChatGpt 4o-mini','Mistral','Qwen2.5','llama3.2'],label='Select a Model:'),
#         gr.Dropdown(['Funny','Formal','Persuasive','Snarky'],label='Select a Tone:'),
#         # gr.Number(label='Enter desired wordlength:'),
#     ],
#     outputs=[gr.Markdown(label='Brochure')],
#     flagging_mode='never'
# )
# view.launch(share=True)